In [5]:
from tqdm import tqdm
from datasets import load_dataset
from pymilvus import MilvusClient, FieldSchema, CollectionSchema, DataType
from sentence_transformers import SentenceTransformer

In [27]:
pip show pymilvus

Name: pymilvus
Version: 2.6.5
Summary: Python Sdk for Milvus
Home-page: 
Author: 
Author-email: Milvus Team <milvus-team@zilliz.com>
License: 
Location: a:\programmers\anaconda3\Lib\site-packages
Requires: grpcio, orjson, pandas, protobuf, python-dotenv, setuptools
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [31]:
pip show tqdm

Name: tqdm
Version: 4.67.1
Summary: Fast, Extensible Progress Meter
Home-page: https://tqdm.github.io
Author: 
Author-email: 
License: MPL-2.0 AND MIT
Location: a:\programmers\anaconda3\Lib\site-packages
Requires: colorama
Required-by: anaconda-client, anaconda-project, conda, conda-build, datasets, huggingface-hub, ir_datasets, mteb, nltk, panel, peft, sentence-transformers, spacy, torchtext, transformers
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip show datasets

Name: datasets
Version: 2.21.0
Summary: HuggingFace community-driven open-source library of datasets
Home-page: https://github.com/huggingface/datasets
Author: HuggingFace Inc.
Author-email: thomas@huggingface.co
License: Apache 2.0
Location: a:\programmers\anaconda3\Lib\site-packages
Requires: aiohttp, dill, filelock, fsspec, huggingface-hub, multiprocess, numpy, packaging, pandas, pyarrow, pyyaml, requests, tqdm, xxhash
Required-by: beir, FlagEmbedding, mteb
Note: you may need to restart the kernel to use updated packages.


In [34]:
pip show sentence_transformers

Name: sentence-transformers
Version: 5.1.1
Summary: Embeddings, Retrieval, and Reranking
Home-page: https://www.SBERT.net
Author: 
Author-email: Nils Reimers <info@nils-reimers.de>, Tom Aarsen <tom.aarsen@huggingface.co>
License: Apache 2.0
Location: a:\programmers\anaconda3\Lib\site-packages
Requires: huggingface-hub, Pillow, scikit-learn, scipy, torch, tqdm, transformers, typing_extensions
Required-by: beir, FlagEmbedding, mteb
Note: you may need to restart the kernel to use updated packages.


In [10]:
from pymilvus import connections

In [4]:
# !pip install tf-keras

In [6]:
embedding_dim = 384
collection_name = "movie_embeddings"

In [7]:
ds = load_dataset("vishnupriyavr/wiki-movie-plots-with-summaries", split="train")
print(ds)

Generating train split:   0%|          | 0/34886 [00:00<?, ? examples/s]

Dataset({
    features: ['Release Year', 'Title', 'Origin/Ethnicity', 'Director', 'Cast', 'Genre', 'Wiki Page', 'Plot', 'PlotSummary'],
    num_rows: 34886
})


In [9]:
# Initialize Milvus client
client = MilvusClient(
    uri="http://localhost:19530",
)

# Create a new schema for the collection
schema = client.create_schema()

In [ ]:
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
    FieldSchema(name="title", dtype=DataType.VARCHAR, max_length=256),
    FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=embedding_dim),
    FieldSchema(name="year", dtype=DataType.INT64),
    FieldSchema(name="origin", dtype=DataType.VARCHAR, max_length=64),
]

schema = CollectionSchema(fields=fields, enable_dynamic_field=False)
client.create_collection(collection_name=collection_name, schema=schema)

In [ ]:
index_params = client.prepare_index_params()
index_params.add_index(field_name="embedding", index_type="FLAT", metric_type="IP") # 使用 FLAT 索引类型和内积距离度量
client.create_index(collection_name, index_params)

在本例中，我们将使用 SentenceTransformers miniLM 模型来创建情节文本的嵌入。该模型可返回 384 维嵌入。

In [15]:
model = SentenceTransformer("all-MiniLM-L12-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

a:\programmers\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\raymo\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L12-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/352 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

我们循环浏览数据行，嵌入情节摘要字段，并将实体插入向量数据库。一般来说，应该像我们这里一样，在成批数据项上执行这一步骤，以最大限度地提高嵌入模型的 CPU 或 GPU 吞吐量。

In [16]:
for batch in tqdm(ds.batch(batch_size=512)):
    embeddings = model.encode(batch["PlotSummary"])
    data = [
        {"title": title, "embedding": embedding, "year": year, "origin": origin}
        for title, embedding, year, origin in zip(
            batch["Title"], embeddings, batch["Release Year"], batch["Origin/Ethnicity"]
        )
    ]
    res = client.insert(collection_name=collection_name, data=data)
    print(res)

Batching examples:   0%|          | 0/34886 [00:00<?, ? examples/s]

  1%|▏         | 1/69 [00:06<07:30,  6.62s/it]

{'insert_count': 512, 'ids': [463093537177603337, 463093537177603338, 463093537177603339, 463093537177603340, 463093537177603341, 463093537177603342, 463093537177603343, 463093537177603344, 463093537177603345, 463093537177603346, 463093537177603347, 463093537177603348, 463093537177603349, 463093537177603350, 463093537177603351, 463093537177603352, 463093537177603353, 463093537177603354, 463093537177603355, 463093537177603356, 463093537177603357, 463093537177603358, 463093537177603359, 463093537177603360, 463093537177603361, 463093537177603362, 463093537177603363, 463093537177603364, 463093537177603365, 463093537177603366, 463093537177603367, 463093537177603368, 463093537177603369, 463093537177603370, 463093537177603371, 463093537177603372, 463093537177603373, 463093537177603374, 463093537177603375, 463093537177603376, 463093537177603377, 463093537177603378, 463093537177603379, 463093537177603380, 463093537177603381, 463093537177603382, 463093537177603383, 463093537177603384, 4630935371

  3%|▎         | 2/69 [00:13<07:29,  6.71s/it]

{'insert_count': 512, 'ids': [463093537177603849, 463093537177603850, 463093537177603851, 463093537177603852, 463093537177603853, 463093537177603854, 463093537177603855, 463093537177603856, 463093537177603857, 463093537177603858, 463093537177603859, 463093537177603860, 463093537177603861, 463093537177603862, 463093537177603863, 463093537177603864, 463093537177603865, 463093537177603866, 463093537177603867, 463093537177603868, 463093537177603869, 463093537177603870, 463093537177603871, 463093537177603872, 463093537177603873, 463093537177603874, 463093537177603875, 463093537177603876, 463093537177603877, 463093537177603878, 463093537177603879, 463093537177603880, 463093537177603881, 463093537177603882, 463093537177603883, 463093537177603884, 463093537177603885, 463093537177603886, 463093537177603887, 463093537177603888, 463093537177603889, 463093537177603890, 463093537177603891, 463093537177603892, 463093537177603893, 463093537177603894, 463093537177603895, 463093537177603896, 4630935371

...

{'insert_count': 512, 'ids': [463093537177637129, 463093537177637130, 463093537177637131, 463093537177637132, 463093537177637133, 463093537177637134, 463093537177637135, 463093537177637136, 463093537177637137, 463093537177637138, 463093537177637139, 463093537177637140, 463093537177637141, 463093537177637142, 463093537177637143, 463093537177637144, 463093537177637145, 463093537177637146, 463093537177637147, 463093537177637148, 463093537177637149, 463093537177637150, 463093537177637151, 463093537177637152, 463093537177637153, 463093537177637154, 463093537177637155, 463093537177637156, 463093537177637157, 463093537177637158, 463093537177637159, 463093537177637160, 463093537177637161, 463093537177637162, 463093537177637163, 463093537177637164, 463093537177637165, 463093537177637166, 463093537177637167, 463093537177637168, 463093537177637169, 463093537177637170, 463093537177637171, 463093537177637172, 463093537177637173, 463093537177637174, 463093537177637175, 463093537177637176, 4630935371

 99%|█████████▊| 68/69 [09:00<00:07,  7.33s/it]

{'insert_count': 512, 'ids': [463093537177637641, 463093537177637642, 463093537177637643, 463093537177637644, 463093537177637645, 463093537177637646, 463093537177637647, 463093537177637648, 463093537177637649, 463093537177637650, 463093537177637651, 463093537177637652, 463093537177637653, 463093537177637654, 463093537177637655, 463093537177637656, 463093537177637657, 463093537177637658, 463093537177637659, 463093537177637660, 463093537177637661, 463093537177637662, 463093537177637663, 463093537177637664, 463093537177637665, 463093537177637666, 463093537177637667, 463093537177637668, 463093537177637669, 463093537177637670, 463093537177637671, 463093537177637672, 463093537177637673, 463093537177637674, 463093537177637675, 463093537177637676, 463093537177637677, 463093537177637678, 463093537177637679, 463093537177637680, 463093537177637681, 463093537177637682, 463093537177637683, 463093537177637684, 463093537177637685, 463093537177637686, 463093537177637687, 463093537177637688, 4630935371

100%|██████████| 69/69 [09:01<00:00,  7.84s/it]

{'insert_count': 70, 'ids': [463093537177638153, 463093537177638154, 463093537177638155, 463093537177638156, 463093537177638157, 463093537177638158, 463093537177638159, 463093537177638160, 463093537177638161, 463093537177638162, 463093537177638163, 463093537177638164, 463093537177638165, 463093537177638166, 463093537177638167, 463093537177638168, 463093537177638169, 463093537177638170, 463093537177638171, 463093537177638172, 463093537177638173, 463093537177638174, 463093537177638175, 463093537177638176, 463093537177638177, 463093537177638178, 463093537177638179, 463093537177638180, 463093537177638181, 463093537177638182, 463093537177638183, 463093537177638184, 463093537177638185, 463093537177638186, 463093537177638187, 463093537177638188, 463093537177638189, 463093537177638190, 463093537177638191, 463093537177638192, 463093537177638193, 463093537177638194, 463093537177638195, 463093537177638196, 463093537177638197, 463093537177638198, 463093537177638199, 463093537177638200, 46309353717

执行搜索

In [17]:
queries = [
    'A shark terrorizes an LA beach.',
    'An archaeologist searches for ancient artifacts while fighting Nazis.',
    'Teenagers in detention learn about themselves.',
    'A teenager fakes illness to get off school and have adventures with two friends.',
    'A young couple with a kid look after a hotel during winter and the husband goes insane.',
    'Four turtles fight bad guys.'
    ]

In [22]:
# Search the database based on input text
def embed_query(data):
    vectors = model.encode(data)
    return [x for x in vectors]


query_vectors = embed_query(queries)

In [23]:
res = client.search(
    collection_name=collection_name,
    data=query_vectors,
    filter='origin == "American" and year > 1945 and year < 2000',
    anns_field="embedding",
    limit=3,
    output_fields=["title"],
)


In [24]:
for idx, hits in enumerate(res):
    print("Query:", queries[idx])
    print("Results:")
    for hit in hits:
        print(hit["entity"].get("title"), "(", round(hit["distance"], 2), ")")
    print()

Query: A shark terrorizes an LA beach.
Results:
Jaws ( 0.6 )
Jaws 3-D ( 0.54 )
Jaws: The Revenge ( 0.54 )

Query: An archaeologist searches for ancient artifacts while fighting Nazis.
Results:
Love Slaves of the Amazons ( 0.4 )
A Time to Love and a Time to Die ( 0.39 )
The Fifth Element ( 0.39 )

Query: Teenagers in detention learn about themselves.
Results:
The Breakfast Club ( 0.54 )
Up the Academy ( 0.46 )
Fame ( 0.43 )

Query: A teenager fakes illness to get off school and have adventures with two friends.
Results:
Ferris Bueller's Day Off ( 0.48 )
Fever Lake ( 0.47 )
Losin' It ( 0.39 )

Query: A young couple with a kid look after a hotel during winter and the husband goes insane.
Results:
The Shining ( 0.48 )
The Four Seasons ( 0.42 )
Highball ( 0.41 )

Query: Four turtles fight bad guys.
Results:
Teenage Mutant Ninja Turtles II: The Secret of the Ooze ( 0.47 )
Devil May Hare ( 0.43 )
Attack of the Giant Leeches ( 0.42 )

